In [9]:
# Basic imports
import sys
import os
sys.path.insert(0, os.path.join(os.getcwd(), '../core'))

import numpy as np
import cupy as cp
import matplotlib.pyplot as plt
import pandas as pd

import kaggle_support as kgs
import pack_cost
import pack_minkowski as mink

print("Imports complete")

Imports complete


In [10]:
# Generate 2000 2-tree test cases
np.random.seed(42)
N_test_cases = 2000
N_trees = 2

# Random positions: (dx, dy) pairs between trees
# Generate relative separations in a reasonable range
# Tree 1 at origin, Tree 2 at various positions
dx = np.random.uniform(-0.5, 0.5, N_test_cases)
dy = np.random.uniform(-0.5, 0.5, N_test_cases)

# Random rotation angles for tree 2 (tree 1 always at theta=0)
theta2 = np.random.uniform(0, 2*np.pi, N_test_cases)

print(f"Generated {N_test_cases} test cases")
print(f"dx range: [{dx.min():.3f}, {dx.max():.3f}]")
print(f"dy range: [{dy.min():.3f}, {dy.max():.3f}]")
print(f"theta2 range: [{theta2.min():.3f}, {theta2.max():.3f}]")

Generated 2000 test cases
dx range: [-0.497, 0.500]
dy range: [-0.500, 0.500]
theta2 range: [0.000, 6.279]


In [11]:
def compute_exact_separation(dx: np.ndarray, dy: np.ndarray, theta2: np.ndarray) -> np.ndarray:
    """
    Compute exact (non-discretized) separation distance using pack_minkowski directly.
    
    Parameters
    ----------
    dx : np.ndarray
        X separation between trees (shape: N,)
    dy : np.ndarray
        Y separation between trees (shape: N,)
    theta2 : np.ndarray
        Rotation angle of tree 2 in radians (shape: N,)
        
    Returns
    -------
    cost : np.ndarray
        Cost values for each test case (shape: N,)
        Applies max(0, separation)^2 transform to raw distances
    """
    N = len(dx)
    costs = np.zeros(N)
    
    # Tree 1 always at origin with rotation 0
    tree1_pos = (0.0, 0.0)
    
    # Process each test case
    # We need to call separation_distance for each unique theta2
    unique_thetas = np.unique(theta2)
    
    for theta in unique_thetas:
        # Find all test cases with this theta
        mask = theta2 == theta
        indices = np.where(mask)[0]
        
        # Get tree2 positions for this theta
        tree2_positions = np.column_stack([dx[mask], dy[mask]])  # (n, 2)
        
        # Compute separation distances (vectorized for this theta)
        sep_distances = mink.separation_distance(tree1_pos, tree2_positions, theta)
        
        # Apply cost transform: max(0, sep)^2
        costs[indices] = np.maximum(0, sep_distances) ** 2
    
    return costs

In [12]:
def compute_discretized_separation(dx: np.ndarray, dy: np.ndarray, theta2: np.ndarray,
                                   Nx: int, Ny: int, Ntheta: int) -> np.ndarray:
    """
    Compute discretized separation distance using CollisionCostExactSeparation with lookup table.
    
    This is a wrapper around pack_cost.CollisionCostExactSeparation that creates
    a lookup table and evaluates the cost.
    
    Parameters
    ----------
    dx : np.ndarray
        X separation between trees (shape: N,)
    dy : np.ndarray
        Y separation between trees (shape: N,)
    theta2 : np.ndarray
        Rotation angle of tree 2 in radians (shape: N,)
    Nx : int
        Number of discretization points in X
    Ny : int
        Number of discretization points in Y
    Ntheta : int
        Number of discretization points in theta
        
    Returns
    -------
    cost : np.ndarray
        Cost values for each test case (shape: N,)
    """
    N = len(dx)
    
    # Create cost function with lookup table
    cost_fn = pack_cost.CollisionCostExactSeparation(
        scaling=1.0,
        use_lookup_table=True,
        lut_N_x=Nx,
        lut_N_y=Ny,
        lut_N_theta=Ntheta,
        lut_trim_zeros=False
    )
    
    # Create a solution collection with 2 trees for each test case
    # Tree 1 at origin with theta=0, Tree 2 at (dx, dy) with theta=theta2
    xyt = cp.zeros((N, 2, 3), dtype=cp.float32)
    xyt[:, 0, :] = cp.array([0.0, 0.0, 0.0], dtype=cp.float32)  # Tree 1 at origin, no rotation
    xyt[:, 1, 0] = cp.asarray(dx, dtype=cp.float32)              # Tree 2 x position
    xyt[:, 1, 1] = cp.asarray(dy, dtype=cp.float32)              # Tree 2 y position
    xyt[:, 1, 2] = cp.asarray(theta2, dtype=cp.float32)          # Tree 2 rotation
    
    # Create solution collection (using square boundary with large size)
    # Boundary parameters don't matter for pairwise collision cost
    h = cp.ones((N, 1), dtype=cp.float32) * 100.0  # Large boundary
    
    solution = kgs.SolutionCollectionSquare(
        xyt=xyt,
        h=h,
        periodic=False
    )
    
    # Compute cost using the proper method
    cost, grad_xyt, grad_h = cost_fn.compute_cost_allocate(solution, evaluate_gradient=False)
    
    # Return only the cost for each test case
    return cost.get()

In [13]:
# Test the functions with a small example
test_dx = dx[:10]
test_dy = dy[:10]
test_theta = theta2[:10]

print("Testing exact separation function...")
exact_costs = compute_exact_separation(test_dx, test_dy, test_theta)
print(f"Exact costs (first 10): {exact_costs}")

print("\nTesting discretized separation function...")
discretized_costs = compute_discretized_separation(test_dx, test_dy, test_theta,
                                                    Nx=100, Ny=100, Ntheta=100)
print(f"Discretized costs (first 10): {discretized_costs}")

print("\nComparison (first 10):")
for i in range(10):
    print(f"  Case {i}: exact={exact_costs[i]:.6f}, discretized={discretized_costs[i]:.6f}, "
          f"error={abs(exact_costs[i] - discretized_costs[i]):.6e}")

Testing exact separation function...
Exact costs (first 10): [0.11919857 0.01883901 0.07535174 0.14145994 0.03419329 0.04022005
 0.03335903 0.04157878 0.05526855 0.09470103]

Testing discretized separation function...
Loading cached lookup table from /mnt/d//packing/temp//lut_cache/exact_sep_Nx100_Ny100_Nt100_trimFalse_tree116323fb.lut_cache...
Successfully loaded cached lookup table
Discretized costs (first 10): [0.11929011 0.01895291 0.0744287  0.14140454 0.03433665 0.04015291
 0.03330911 0.04156759 0.05534661 0.09471458]

Comparison (first 10):
  Case 0: exact=0.119199, discretized=0.119290, error=9.153250e-05
  Case 1: exact=0.018839, discretized=0.018953, error=1.138945e-04
  Case 2: exact=0.075352, discretized=0.074429, error=9.230428e-04
  Case 3: exact=0.141460, discretized=0.141405, error=5.539886e-05
  Case 4: exact=0.034193, discretized=0.034337, error=1.433533e-04
  Case 5: exact=0.040220, discretized=0.040153, error=6.713075e-05
  Case 6: exact=0.033359, discretized=0.0333